In [3]:
from glob import glob
from streaming import MDSWriter
from streaming import StreamingDataset
from transformers import default_data_collator
from tqdm import tqdm
import numpy as np

In [5]:
!rm -rf nanot5-512

In [6]:
folders = glob('nanot5-*')
folders = sorted(folders, key = lambda x: int(x.split('-')[-1]))
folders

['nanot5-0',
 'nanot5-1',
 'nanot5-2',
 'nanot5-3',
 'nanot5-4',
 'nanot5-5',
 'nanot5-6',
 'nanot5-7',
 'nanot5-8',
 'nanot5-9',
 'nanot5-10',
 'nanot5-11',
 'nanot5-12',
 'nanot5-13',
 'nanot5-14',
 'nanot5-15',
 'nanot5-16',
 'nanot5-17',
 'nanot5-18',
 'nanot5-19',
 'nanot5-20',
 'nanot5-21',
 'nanot5-22',
 'nanot5-23',
 'nanot5-24',
 'nanot5-25',
 'nanot5-26',
 'nanot5-27',
 'nanot5-28',
 'nanot5-29']

In [7]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class Int32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.int32)

_encodings['int32'] = Int32

In [8]:
columns = {
    'input_ids': 'int32',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [9]:
!rm -rf nanot5-512

In [10]:
with MDSWriter(out='nanot5-512', columns=columns, compression=compression, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = StreamingDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 3798276/3798276 [03:12<00:00, 19712.86it/s]


In [11]:
%%time
dataset = StreamingDataset(local='nanot5-512')

CPU times: user 312 ms, sys: 96.7 ms, total: 409 ms
Wall time: 406 ms


In [13]:
dataset[0]['input_ids'].shape

(568,)